In [203]:
from Operations.CO_CompareMinAMI import CO_CompareMinAMI
from Operations.CO_AutoCorrShape import CO_AutoCorrShape
from Operations.CO_TranslateShape import CO_TranslateShape
from Operations.CP_wavelet_varch import CP_wavelet_varchg
from Operations.EN_SampEn import EN_SampEn
from Operations.EN_wentropy import EN_wentropy
from Operations.CO_AddNoise import CO_AddNoise
from Operations.CO_AutoCorr import CO_AutoCorr
from PeripheryFunctions.BF_iszscored import BF_iszscored
from Operations.DT_IsSeasonal import DT_IsSeasonal
from PeripheryFunctions.BF_Embed import BF_Embed
from Operations.EN_rpde import EN_rpde
from Operations.SC_FluctAnal import SC_FluctAnal
from Operations.CO_CompareMinAMI import CO_CompareMinAMI
from Operations.PH_ForcePotential import PH_ForcePotential
from PeripheryFunctions.BF_zscore import BF_zscore
from PeripheryFunctions.BF_SimpleBinner import BF_SimpleBinner
import numpy as np
import warnings
from sklearn.neighbors import KernelDensity
from loguru import logger
from Utils import wrcoef
from scipy.interpolate import make_lsq_spline, BSpline
from Utils.wrcoef import wavedec, wrcoef, detcoef
from Operations.WL_coeffs import WL_coeffs
import pywt
from typing import Union, Optional
import numpy.linalg
from lmfit.models import GaussianModel, PowerLawModel, ExponentialModel
from statsmodels.tsa.ar_model import AutoReg, ar_select_order, AR

In [4]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")
ts4 = np.loadtxt("ts4.txt")

In [21]:
kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(ts1[:, None]) # closest match to the default behaviour of ksdensity in MATLAB
lower = ts1.min() - 3 * kde.bandwidth_
upper = ts1.max() + 3 * kde.bandwidth_
xi = np.linspace(lower, upper, 100)[:,None]
f = np.exp(kde.score_samples(xi))
f

array([0.00193995, 0.00295549, 0.00442302, 0.00650255, 0.00939184,
       0.01332756, 0.01858304, 0.02546165, 0.0342848 , 0.04537408,
       0.05902781, 0.07549287, 0.09493363, 0.11740099, 0.14280473,
       0.17089318, 0.20124363, 0.23326654, 0.26622495, 0.299269  ,
       0.33148362, 0.36194524, 0.38978253, 0.4142349 , 0.43470273,
       0.45078434, 0.46229597, 0.46927332, 0.4719554 , 0.47075333,
       0.46620836, 0.45894416, 0.44961864, 0.43887982, 0.42732933,
       0.41549575, 0.40381836, 0.39264091, 0.38221389, 0.37270314,
       0.36420264, 0.3567493 , 0.3503379 , 0.34493494, 0.34049053,
       0.33694795, 0.33425095, 0.3323489 , 0.33120029, 0.33077496,
       0.33105539, 0.33203736, 0.33373009, 0.3361559 , 0.33934909,
       0.34335395, 0.34822135, 0.35400356, 0.36074694, 0.36848229,
       0.37721285, 0.38690031, 0.39744982, 0.40869516, 0.42038603,
       0.43217949, 0.44363783, 0.45423501, 0.46337295, 0.47040832,
       0.47468902, 0.47559819, 0.47260222, 0.46529811, 0.45345

In [220]:
-2 * np.log10(10954.481) + 5 * np.log10(995)

np.float64(6.90993179085722)

In [170]:
mod = AutoReg(ts1, lags=5, trend='n')
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                     AutoReg(5)   Log Likelihood               10954.481
Method:               Conditional MLE   S.D. of innovations              0.000
Date:                Sat, 15 Feb 2025   AIC                         -21896.962
Time:                        16:05:37   BIC                         -21867.545
Sample:                             5   HQIC                        -21885.779
                                 1000                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
y.L1           0.7647      0.030     25.671      0.000       0.706       0.823
y.L2           0.4847      0.038     12.792      0.000       0.410       0.559
y.L3           0.1460      0.041      3.595      0.000       0.066       0.226
y.L4          -0.1892      0.038     -5.001      0.000      -0.263      -0.115
y.L5          -0.3415      0.030    -11.479      0.000      -0.400      -0.283
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.9801           -0.1987j            1.0000           -0.0318
AR.2            0.9801           +0.1987j            1.0000            0.0318
AR.3           -0.5147           -1.3066j            1.4044           -0.3097
AR.4           -0.5147           +1.3066j            1.4044            0.3097
AR.5           -1.4847           -0.0000j            1.4847           -0.5000
-----------------------------------------------------------------------------
"""

In [40]:
mod = AutoReg(ts1, lags=np.arange(1, 11), trend='n')
res = mod.fit()
res.sigma2

np.float64(1.0500092115061956e-11)

In [142]:
eigvals, eigvecs = np.linalg.eig(np.diag(res.params))

In [110]:
CO_AutoCorr(res.resid, list(np.arange(1, 21)), 'Fourier')

array([-0.06004257, -0.07960433, -0.11872864, -0.13083671, -0.12413532,
       -0.08097929, -0.06575356,  0.01040791,  0.07933451,  0.17865283,
       -0.03238517,  0.01132411, -0.06359846,  0.00245538,  0.04774069,
        0.02427306, -0.02749503,  0.02765683, -0.01020147, -0.02863997])

In [92]:
res.conf_int()

array([[ 0.27902981,  0.39968788],
       [ 0.20926432,  0.33651149],
       [ 0.11079878,  0.24163217],
       [ 0.01854654,  0.1510526 ],
       [ 0.09347442,  0.22639271],
       [-0.05131512,  0.0815639 ],
       [-0.11812205,  0.01439032],
       [-0.17672288, -0.04595361],
       [-0.16130575, -0.03431713],
       [-0.30772463, -0.18724772]])

In [163]:
res.llf

np.float64(11108.671209667926)

In [155]:
(res.bic)

np.float64(-22141.467664961438)

In [160]:
res.nobs

990

In [158]:
def arfit(x, pmin, pmax):

    N = len(x)

    mod = AutoReg(x, lags=np.arange(pmin, pmax+1), trend='n') # no intercept vector (data assumed to be zero-mean, zscored)
    mod_fit = mod.fit()
    aest = mod_fit.params # AR model coefficients
    cest = mod_fit.sigma2 # noise variance

    popt = len(aest)

    out = {}
    out["A1"] = aest[0]
    for i in range(1, 6):
        if popt >= i:
            out[f"A{i+1}"] = aest[i]
        else:
            # if it's as if the higher order coefficients are all zero
            out[f"A{i}"] = 0

    # summary statistics on the coefficients
    out["maxA"] = np.max(aest)
    out["minA"] = np.min(aest)
    out["meanA"] = np.mean(aest)
    out["stdA"] = np.std(aest, ddof=1)
    out["sumA"] = np.sum(aest)
    out["rmsA"] = np.sqrt(np.sum(aest**2))
    out["sumsqA"] = np.sum(aest**2)

    # noise covariance matrix
    out["C"] = cest

    # Bayesian Information Criterion (BIC)
    

    # Aikiake

    # Test Resdiuals
    residuals = mod_fit.resid
    out["res_ac1"] = CO_AutoCorr(residuals, 1, 'Fourier')
    out["res_ac1_norm"] = CO_AutoCorr(residuals, 1, 'Fourier')/np.sqrt(N) # normalize by sqrt(N)
    # calculate correlations up to 20, return how many exceed significance threshold
    acf = CO_AutoCorr(residuals, list(np.arange(1, 21)), 'Fourier') 
    out['pcorr_res'] = np.sum(np.abs(acf)>1.96/np.sqrt(N))/20

    # Confidence intervals
    ci95s = res.bse * 1.96 # 95% CI
    out["aerr_min"] = np.min(ci95s)
    out["aerr_max"] = np.max(ci95s)
    out["aerr_mean"] = np.mean(ci95s)

    # eigendecomposition
    

    return out


In [159]:
arfit(ts1, 1, 10)

TypeError: AutoReg.__init__() got an unexpected keyword argument 'method'

In [239]:
def MF_ExpSmoothing(x, ntrain : Optional[float] = None , alpha : str = 'best'):
    N = len(x)
    if not ntrain:
        ntrain = min(100, N)
    if (ntrain > 0) and (ntrain < 1):
        ntrain = int(np.floor(N*ntrain))
    
    out = {}
    minTrain = 100
    maxTrain = 1000

    if ntrain > maxTrain:
        print(f"Training set size reduced from {ntrain} to maximum of 1000 samples.")
        ntrain = 1000
    if ntrain < minTrain:
        print(f"Training set size increased from {ntrain} to minimum of 100.")
        ntrain = 100
    if N < ntrain:
        logger.warning("Time Series too short for exponential smoothing")
        return np.nan
    
    if alpha == 'best':
        xtrain = x[:ntrain]
        # do a descent
        # fits a quadratic to available points
        alphar = np.linspace(0.1, 0.9, 5)
        rmses = np.zeros(len(alphar))
        for (i, a) in enumerate(alphar):
            xf = SUB_fit_exp_smooth(xtrain, a)
            # issue forecasts
            fore = xf[2:]
            orig = xtrain[2:]
            rmses[i] = np.sqrt(np.mean((fore - orig)**2))

        # fit quadratic to set alpha
        ix = np.argsort(rmses)
        rkeep = ix[:3]
        p = np.polyfit(alphar[rkeep], rmses[rkeep], 2)
        aar = np.arange(0, 1.005, 0.005)
        y = np.polyval(p, aar)
        alphamin = -p[1]/(2*p[0])
        out['alphamin_1'] = alphamin
        out['p1_1'] = np.abs(p[0]) # concavity
        out['cup_1'] = np.sign(p[0])

        if p[0] < 0:  # concave down -- it's looking at a minimum
            if y[0] < y[-1]:
                alphamin = 0.01
            else:
                alphamin = 1
        else:
            #% Search again around this
            alphar = np.linspace(alphamin-0.1, alphamin+0.1, 5)
            if np.any(alphar <= 0):
                alphar = np.linspace(0.1, max(alphamin, 0)+0.1, 5)
            elif any(alphar >= 1):
                alphar = np.linspace(min(alphamin, 1)-0.1, 1, 5)
            
            for k in range(len(alphar)):
                a = alphar[k]
                xf = SUB_fit_exp_smooth(xtrain, a)
                fore = xf[2:]
                orig = xtrain[2:]
                rmses[k] = np.sqrt(np.mean((fore - orig)**2))
            
            p = np.polyfit(alphar, rmses, 2)

            if p[0] < 0:
                alphamin = alphar[np.argmin(rmses)]
            else:
                alphamin = -p[1]/(2*p[0])
                if alphamin > 1:
                    alphamin = 1
                elif alphamin <= 0:
                    alphamin = 0.01
        
    out['alphamin'] = alphamin
    alpha = alphamin

    #(2) Fit to the whole time series
    y = SUB_fit_exp_smooth(x, alpha)
    yp = y[2:N] # predicted
    xp = x[2:N] # original
    e = yp-xp # residuals

    # stats on residuals
    out['meane'] = np.mean(e)
    out['meanabs'] = np.mean(np.abs(e))
    out['rmse'] = np.sqrt(np.mean(e**2))
    out['stde'] = np.std(e, ddof=1)
    out['mms'] = np.abs(np.mean(e)) + np.abs(np.std(e, ddof=1))
    out['maxonmean'] = np.max(e)/np.abs(np.mean(e))

    if np.std(e, ddof=1) == 0:
        e = np.zeros(len(e))
    else:
        e = BF_zscore(e)
    
    # Identify any low-frequency trends in residuals

    # Analyze autocorrelation in residuals
    maxLag = 25
    autoCorrResid = CO_AutoCorr(e, list(np.arange(1, maxLag+1)), 'Fourier')
    sqrtN = np.sqrt(N)
    out['ac1'] = autoCorrResid[0]
    out['ac2'] = autoCorrResid[1]
    out['ac3'] = autoCorrResid[2]
    out['ac1n'] = autoCorrResid[0]*sqrtN
    out['ac2n'] = autoCorrResid[1]*sqrtN
    out['ac3n'] = autoCorrResid[2]*sqrtN

    # Median normalized distance from zero
    out['acmnd0'] = np.median(np.abs(autoCorrResid))*sqrtN 
    out['acsnd0'] = np.std(np.abs(autoCorrResid), ddof=1)*sqrtN
    out['propbth'] = np.sum(np.abs(autoCorrResid) < 2.6/sqrtN)/maxLag

    # First time to get below the significance threshold
    ftbth = np.argwhere(np.abs(autoCorrResid) < 2.6/sqrtN).flatten()[0]
    if not ftbth:
        ftbth = maxLag + 1
    
    out['ftbth'] = ftbth
    #% Durbin-Watson test statistic (like AC1)
    out['dwts'] = np.sum((e[1:] - e[:-1])**2) / np.sum(e**2)


    return out


def SUB_fit_exp_smooth(x, a):
    ntrain = len(x)
    xf = np.zeros(ntrain)
    
    for i in range(1, ntrain - 1):
        s = np.zeros(i + 1)
        s[0] = np.mean(x[0:i])
        
        # Smooth data within the window
        for j in range(1, i + 1):
            s[j] = a * x[j] + (1 - a) * s[j - 1]
        
        #The forecast for x[i+1] is taken as the last smoothed value s[i]
        xf[i + 1] = s[i]
    
    return xf
    

In [240]:
res = MF_ExpSmoothing(ts1, 0.5)
res

{'alphamin_1': np.float64(1.0901076386948796),
 'p1_1': np.float64(0.3665076982852078),
 'cup_1': np.float64(1.0),
 'alphamin': 1,
 'meane': np.float64(0.001265250501002004),
 'meanabs': np.float64(0.1267746092184369),
 'rmse': np.float64(0.14090634055511642),
 'stde': np.float64(0.14097130446993963),
 'mms': np.float64(0.14223655497094165),
 'maxonmean': np.float64(157.8074854282818),
 'ac1': np.float64(0.9789764444623205),
 'ac2': np.float64(0.919003505565848),
 'ac3': np.float64(0.8225485607199285),
 'ac1n': np.float64(30.95795340154266),
 'ac2n': np.float64(29.06144255267308),
 'ac3n': np.float64(26.011269379682837),
 'acmnd0': np.float64(20.24163306315739),
 'acsnd0': np.float64(10.142573846880763),
 'propbth': np.float64(0.04),
 'ftbth': np.int64(7),
 'dwts': np.float64(0.040146836050223816)}

In [195]:
CO_AutoCorr(e, taus, 'Fourier')

array([ 0.04891096,  0.12065962,  0.03132307, -0.18413738, -0.23652705,
       -0.36872942, -0.29583703, -0.1893701 , -0.11025447,  0.11082915,
        0.20518548,  0.33283003,  0.33624639,  0.25478703,  0.11958142,
       -0.08063441, -0.16793048, -0.28780988, -0.36458256, -0.24298984,
       -0.1825511 ,  0.01683114,  0.15704877,  0.26346958,  0.31175132])

In [111]:
np.sort([0.1, 0.3, 0.4, 0.1, 0.004])

array([0.004, 0.1  , 0.1  , 0.3  , 0.4  ])

In [115]:
aar = np.arange(0, 1.005, 0.005)
aar

array([0.   , 0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 ,
       0.045, 0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085,
       0.09 , 0.095, 0.1  , 0.105, 0.11 , 0.115, 0.12 , 0.125, 0.13 ,
       0.135, 0.14 , 0.145, 0.15 , 0.155, 0.16 , 0.165, 0.17 , 0.175,
       0.18 , 0.185, 0.19 , 0.195, 0.2  , 0.205, 0.21 , 0.215, 0.22 ,
       0.225, 0.23 , 0.235, 0.24 , 0.245, 0.25 , 0.255, 0.26 , 0.265,
       0.27 , 0.275, 0.28 , 0.285, 0.29 , 0.295, 0.3  , 0.305, 0.31 ,
       0.315, 0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 , 0.355,
       0.36 , 0.365, 0.37 , 0.375, 0.38 , 0.385, 0.39 , 0.395, 0.4  ,
       0.405, 0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 , 0.445,
       0.45 , 0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485, 0.49 ,
       0.495, 0.5  , 0.505, 0.51 , 0.515, 0.52 , 0.525, 0.53 , 0.535,
       0.54 , 0.545, 0.55 , 0.555, 0.56 , 0.565, 0.57 , 0.575, 0.58 ,
       0.585, 0.59 , 0.595, 0.6  , 0.605, 0.61 , 0.615, 0.62 , 0.625,
       0.63 , 0.635,

In [9]:
gmodel = GaussianModel()
gmodel.guess(ts1)

TypeError: GaussianModel.guess() missing 1 required positional argument: 'x'

In [28]:
dny, binEdges = BF_SimpleBinner(ts1, 10)
dnx = np.mean([binEdges[:-1]])
dnx

np.float64(-0.09999450000000012)

In [62]:
a = binEdges[:-1]
b = binEdges[1:]
stacked = np.vstack((a, b))
np.mean(stacked, axis=0)

array([-0.8999905, -0.6999915, -0.4999925, -0.2999935, -0.0999945,
        0.1000045,  0.3000035,  0.5000025,  0.7000015,  0.9000005])

In [40]:
np.hstack([binEdges[:-1], binEdges[2:-1]])

array([-9.99990e-01, -7.99991e-01, -5.99992e-01, -3.99993e-01,
       -1.99994e-01,  5.00000e-06,  2.00004e-01,  4.00003e-01,
        6.00002e-01,  8.00001e-01, -5.99992e-01, -3.99993e-01,
       -1.99994e-01,  5.00000e-06,  2.00004e-01,  4.00003e-01,
        6.00002e-01,  8.00001e-01])

In [524]:
def findMyThreshold(x, det_s, N):
    pr = np.argwhere(det_s < x*np.max(det_s)).flatten()
    if len(pr) == 0:
        return np.nan
    else:
        return pr[0]/N

def WL_coeffs(y : list, wname : str = 'db3', level : int = 3):
    N = len(y)
    if level == 'max':
        level = pywt.dwt_max_level(N, wname)
        if level == 0:
            raise ValueError("Cannot compute wavelet coefficients (short time series)")
    
    if pywt.dwt_max_level(N, wname) < level:
        raise ValueError(f"Chosen level, {level}, is too large for this wavelet on this signal.")
    
    #coeffs = pywt.wavedec(y, wname, level=level)
    C, L = wavedec(y, wavelet=wname, level=level)
    det = wrcoef(C, L, wname, level)
    det_s = np.sort(np.abs(det))[::-1]

    #%% Return statistics
    out = {}
    out['mean_coeff'] = np.mean(det_s)
    out['max_coeff'] = np.max(det_s)
    out['med_coeff'] = np.median(det_s)

    #% Decay rate stats ('where below _ maximum' = 'wb_m')
    out['wb99m'] = findMyThreshold(0.99, det_s, N)
    out['wb90m'] = findMyThreshold(0.90, det_s, N)
    out['wb75m'] = findMyThreshold(0.75, det_s, N)
    out['wb50m'] = findMyThreshold(0.50, det_s, N)
    out['wb25m'] = findMyThreshold(0.25, det_s, N)
    out['wb10m'] = findMyThreshold(0.10, det_s, N)
    out['wb1m'] = findMyThreshold(0.01, det_s, N)

    return out

In [525]:
WL_coeffs(ts1, 'db3')

{'mean_coeff': np.float64(0.09842152327706816),
 'max_coeff': np.float64(0.2954158069152821),
 'med_coeff': np.float64(0.09023773051023212),
 'wb99m': np.float64(0.002),
 'wb90m': np.float64(0.002),
 'wb75m': np.float64(0.044),
 'wb50m': np.float64(0.223),
 'wb25m': np.float64(0.585),
 'wb10m': np.float64(0.845),
 'wb1m': np.float64(0.983)}

In [438]:
def SY_LocalDistributions(y : list, numSegs : int = 5, eachOrPar : str = 'par', numPoints : int = 200):
    N = len(y)
    lseg = np.floor(N/numSegs)
    dns = np.zeros((numPoints, numSegs))
    r = np.linspace(np.min(y), np.max(y), numPoints)
    
    for i in range(numSegs):
        fit_to = y[int((i)*lseg+1):int((i+1)*lseg)]
        kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(fit_to[:, None])
        dns[:, i] = np.exp(kde.score_samples(r[:, None]))

    if eachOrPar in ['par', 'parent']:
        #% Compares each subdistribtuion to the parent (full signal) distribution
        kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(y[:, None])
        pardn = np.exp(kde.score_samples(r[:, None]))
        divs = np.zeros(numSegs)
        for i in range(numSegs):
            divs[i] = np.sum(np.abs(dns[:, i]-pardn))

    elif eachOrPar == 'each':
        #% Compares each subdistribtuion to the parent (full signal) distribution
        if numSegs == 2:
            out = np.sum(np.abs(dns[:, 0]-dns[:, 1]))
            return out
        diffmat = np.nan * np.empty((numSegs, numSegs,))
        for i in range(numSegs):
            for j in range(numSegs):
                diffmat[i, j] = np.sum(np.abs(dns[:, i]-dns[:,j]))
    
        divs = diffmat[np.triu_indices_from(diffmat, k=1)]
    else:
        raise ValueError(f"Unknown method {eachOrPar} should be 'each' or 'par'")

    #% Return basic statistics on differences in distributions in different segments of the time series
    out = {}
    out['meandiv'] = np.mean(divs)
    out['mediandiv'] = np.median(divs)
    out['mindiv'] = np.min(divs)
    out['maxdiv'] = np.max(divs)
    out['stddiv'] = np.std(divs, ddof=1)
    
    return out


In [441]:
SY_LocalDistributions(ts1, 10)

{'meandiv': np.float64(9.533297079841278),
 'mediandiv': np.float64(9.232399692106519),
 'mindiv': np.float64(8.532618509422912),
 'maxdiv': np.float64(11.237563242946587),
 'stddiv': np.float64(0.993290702736522)}

In [66]:
def NL_fnn(y, maxdim = 10, tau = 1, th = 5, kth = 1, justBest = 0, bestp = 0.1):
    dim = np.arange(1, maxdim+1)
    p = dimension.fnn(y, dim=dim, tau=tau, R=th, window=kth)[0]
    if justBest:
        out = dim[np.argwhere(p < bestp)[0]]
    else:
        out = {}
        for i in range(maxdim):
            out[f"pfnn_{i+1}"] = p[i]

        out['meanpfnn'] = np.mean(p)
        out['stdpfnn'] = np.std(p, ddof=1)

        # find embedding dimension for the first time p goes under x%
        out['firstunder02'] = dim[np.argwhere(p < 0.2)[0]][0]
        out['firstunder01'] = dim[np.argwhere(p < 0.1)[0]][0]
        out['firstunder005'] = dim[np.argwhere(p < 0.05)[0]][0]
        out['firstunder002'] = dim[np.argwhere(p < 0.02)[0]][0]
        out['firstunder001'] = dim[np.argwhere(p < 0.01)[0]][0]

        out['max1stepchange'] = np.max(np.abs(np.diff(p)))

    return out


In [33]:
dimension.fnn(ts3, dim=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], tau=1, R=5,window=1)

array([[0.98498498, 0.78657315, 0.44032096, 0.15963855, 0.05226131,
        0.01307847, 0.0020141 , 0.        , 0.        , 0.        ],
       [0.16816817, 0.15531062, 0.16950853, 0.18072289, 0.19497487,
        0.2444668 , 0.30110775, 0.36693548, 0.49243189, 0.61616162],
       [0.98498498, 0.78757515, 0.45235707, 0.23694779, 0.20301508,
        0.24647887, 0.30110775, 0.36693548, 0.49243189, 0.61616162]])

In [161]:
def DN_OutlierInclude(y, thresholdHow : str = 'abs', inc : float = 0.01):
    #% If the time series is a constant causes issues
    if np.all(y[1] == y):
        # % This method is not suitable for such time series: return a NaN
        warnings.warn("The time series is a constant!")
        return np.nan
    
    # % Check z-scored time series
    if not BF_iszscored(y):
        warnings.warn("The input time series should be z-scored")
    N = len(y)
    
    #%% Initialize thresholds
    # % ------------------------------------------------------------------------------
    # % Could be better to just use a fixed number of increments here, from 0 to the max.
    # % (rather than forcing a fixed inc)
    if thresholdHow == 'abs': # % analyze absolute value deviations
        thr = np.arange(0, max(abs(y)), inc)
        tot = N
    elif thresholdHow == 'pos':
        thr = np.arange(0, max(y), inc)
        tot = sum(y >= 0)
    elif thresholdHow == 'neg':
        thr = np.arange(0, max(-y), inc)
        tot = sum(y <= 0)
    else:
        raise ValueError(f"Error thresholding with '{thresholdHow}'. Must select either 'abs', 'pos', or 'neg'.")
    
    if len(thr) == 0:
        raise ValueError("Error setting increments through the time-series values...")
    
    msDt = np.zeros((len(thr), 6))

    # % Calculate statistics of over-threshold events, looping over thresholds
    for (i, th) in enumerate(thr):
        if thresholdHow == 'abs':
            r = np.argwhere(abs(y >= th)).flatten()
        elif thresholdHow == 'pos':
            r = np.argwhere(y >= th).flatten()
        elif thresholdHow == 'neg':
            r = np.argwhere(y <= -th).flatten()
        
        Dt_exc = np.diff(r)
        msDt[i, 0] = np.mean(Dt_exc)
        msDt[i, 1] = np.std(Dt_exc, ddof=1)/np.sqrt(len(r))
        msDt[i, 2] = len(Dt_exc)/tot*100
        msDt[i, 3] = (np.median(r)/(N/2)) - 1
        msDt[i, 4] = np.mean(r)/(N/2) - 1
        msDt[i, 5] = np.std(r, ddof=1)/np.sqrt(len(r))

    #%% Trim
    fbi = np.argmax(np.isnan(msDt[:, 0])) if np.any(np.isnan(msDt[:, 0])) else None
    if fbi:
        msDt = msDt[:fbi, :]
        thr = thr[:fbi]
    trimthr = 2 
    mj_indices = np.where(msDt[:, 2] > trimthr)[0]
    if mj_indices.size > 0:
        mj = mj_indices[-1]  # last index
        # In MATLAB: msDt = msDt(1:mj,:); we slice up to mj+1 in Python
        msDt = msDt[:mj+1, :]
        thr = thr[:mj+1]




    return fbi


In [175]:
msDt = np.array([
    [4, 5],
    [5, 6]
])

In [176]:
fbi = np.argmax(np.isnan(msDt[:, 0])) if np.any(np.isnan(msDt[:, 0])) else None

In [177]:
if not fbi:
    print("empty")

empty


In [169]:
fbi

np.int64(0)

In [ ]:
np.isnan()

In [163]:
msDt = DN_OutlierInclude(ts1, 'neg')

/var/folders/pt/9v0934pn6s1g4klqjbcz65p40000gp/T/ipykernel_50058/3526833644.py:10: UserWarning: The input time series should be z-scored
  warnings.warn("The input time series should be z-scored")


In [156]:
msDt[1, :]

array([ 1.98787879e+00,  1.71847328e-01,  9.95975855e+01, -1.00000000e-03,
        1.06854839e-02,  1.29149570e+01])

In [106]:
DN_pleft(ts4, 0.4)

np.float64(1.0395880299049998)

In [253]:
def DN_CompareKSFit(x : list, whatDistn : str = 'norm') -> dict:

    # Fit the distribution 'whatDistn' to the input data, x.
    xStep = np.std(x, ddof=1)/100 # set a step size
    if whatDistn == 'norm':
        (a, b) = stats.norm.fit(x)
        peaky = stats.norm.pdf(a, a, b)
        thresh = peaky/100
        xf1 = np.mean(x)
        ange = 10
        while ange > thresh:
            xf1 = xf1 - xStep
            ange = stats.norm.pdf(xf1, a, b)
        xf2 = np.mean(x)
        ange = 10
        while ange > thresh:
            xf2 = xf2 + xStep
            ange = stats.norm.pdf(xf2, a, b)
        xf = [xf1, xf2]

    # Estimate smoothed empirical distribution
    kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(x[:, None]) # closest match to the default behaviour of ksdensity in MATLAB
    lower = x.min() - 3 * kde.bandwidth_
    upper = x.max() + 3 * kde.bandwidth_
    xi = np.linspace(lower, upper, 100)[:,None]
    f = np.exp(kde.score_samples(xi))
    xi = xi[f > 1E-6]
    if len(xi) == 0:
        return np.nan
    
    xi = [np.floor(xi[0]*10)/10, np.ceil(xi[-1]*10)/10]

    # Find appropriate range [x1 x2] that incorporates the full range of both
    x1 = min(xf[0], xi[0])
    x2 = max(xf[1], xi[1])

    xi = np.linspace(x1, x2, 1000)[:, None]
    kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(x[:, None])
    f = np.exp(kde.score_samples(xi))
    ffit = stats.norm.pdf(xi, a, b)

    adiff = np.sum(abs(f - ffit)*(xi[1] - xi[0]))
    max1 = np.max(f)
    max2 = np.max(ffit)
    peaksepy = max2-max1

    i1 = np.argmax(f)
    i2 = np.argmax(ffit)
    peaksepx = xi[i2] - xi[i1]

    olapint = sum(sum(f * ffit) * (xi[1] - xi[0])) * np.std(x, ddof=1)


    return peaksepy, peaksepx, olapint



In [254]:
DN_CompareKSFit(ts1)

(np.float64(0.08766587476060284),
 array([-0.77181288]),
 np.float64(163.92612892310967))

In [88]:
kde = KernelDensity(bandwidth='silverman', kernel='gaussian').fit(ts1[:, None])

In [129]:
lower = ts1.min() - 3 * kde.bandwidth_
upper = ts1.max() + 3 * kde.bandwidth_

In [136]:
np.exp(kde.score_samples(np.linspace(lower, upper, 100)[:,None]))

array([0.00193995, 0.00295549, 0.00442302, 0.00650255, 0.00939184,
       0.01332756, 0.01858304, 0.02546165, 0.0342848 , 0.04537408,
       0.05902781, 0.07549287, 0.09493363, 0.11740099, 0.14280473,
       0.17089318, 0.20124363, 0.23326654, 0.26622495, 0.299269  ,
       0.33148362, 0.36194524, 0.38978253, 0.4142349 , 0.43470273,
       0.45078434, 0.46229597, 0.46927332, 0.4719554 , 0.47075333,
       0.46620836, 0.45894416, 0.44961864, 0.43887982, 0.42732933,
       0.41549575, 0.40381836, 0.39264091, 0.38221389, 0.37270314,
       0.36420264, 0.3567493 , 0.3503379 , 0.34493494, 0.34049053,
       0.33694795, 0.33425095, 0.3323489 , 0.33120029, 0.33077496,
       0.33105539, 0.33203736, 0.33373009, 0.3361559 , 0.33934909,
       0.34335395, 0.34822135, 0.35400356, 0.36074694, 0.36848229,
       0.37721285, 0.38690031, 0.39744982, 0.40869516, 0.42038603,
       0.43217949, 0.44363783, 0.45423501, 0.46337295, 0.47040832,
       0.47468902, 0.47559819, 0.47260222, 0.46529811, 0.45345